In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
from transformers import BertTokenizerFast, BertForTokenClassification
import torch.utils.data as data
import torch
from tqdm import tqdm

In [3]:
url = "https://drive.google.com/file/d/1cxg7dKxIBDtaqn9kD4b3XeboTwOKADlJ/view?usp=sharing"
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)
sent = df["text"].values[0]
print(sent)

Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .


In [4]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")
text_tokenized = tokenizer(sent, padding="max_length", max_length=512, truncation=True, return_tensors="pt")
print(text_tokenized)

{'input_ids': tensor([[  101, 26159,  1104,  8568,  4487,  5067,  1138,  9639,  1194,  1498,
          1106,  5641,  1103,  1594,  1107,  5008,  1105,  4555,  1103, 10602,
          1104,  1418,  2830,  1121,  1115,  1583,   119,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [5]:
tokenizer.decode(text_tokenized["input_ids"][0])

'[CLS] Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [6]:
tokens = tokenizer.convert_ids_to_tokens(text_tokenized["input_ids"][0])
word_ids = text_tokenized.word_ids()
print(tokens)
print(word_ids)
### sub-word tokens + special tokens
### corresponding word idx in the originial sentence

['[CLS]', 'Thousands', 'of', 'demons', '##tra', '##tors', 'have', 'marched', 'through', 'London', 'to', 'protest', 'the', 'war', 'in', 'Iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'British', 'troops', 'from', 'that', 'country', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PA

In [7]:
def get_data_label_details(df):
    
    labels = [label.split() for label in df["labels"].values.tolist()]
    unique_labels = set()
    for sent_label in labels:
        [unique_labels.add(token_lb) for token_lb in sent_label]
    
    num_unique_labels = len(unique_labels)
    print(f"Number of Unique Labels: {num_unique_labels}")
    label_to_idx = { label : idx for idx, label in enumerate(sorted(unique_labels))}
    idx_to_label = { idx : label for idx, label in enumerate(sorted(unique_labels))}
    return num_unique_labels, label_to_idx, idx_to_label

def align_label(tokenized_sent, labels, label_to_idx):
    word_ids = tokenized_sent.word_ids()
    previous_word_idx = None
    label_ids = []
    
    for word_idx in word_ids:
        if word_idx is None: ### special token
            label_ids.append(-100)
        elif word_idx != previous_word_idx: ### new token
            try:
                label_ids.append(label_to_idx[labels[word_idx]])
            except:
                ### not in vocabulary
                label_ids.append(-100)
        else: ### repeated token from the same word
            label_ids.append(-100)
        
        previous_word_idx = word_idx
        
    return label_ids
            
    

In [8]:
class NERDataset(data.Dataset):
    
    def __init__(self, filepath, tokenizer, start=None, end=None):
        df = pd.read_csv(filepath)
        self.num_unique_labels, self.label_to_idx, self.idx_to_label = get_data_label_details(df)
        if(start is not None and end is not None):
            df = df[start:end]
        labels = [label.split() for label in df['labels'].values.tolist()] ### list of lists where each list of the NER labels
        sentences = df["text"].values.tolist() ### list of sentences
        self.txt = [tokenizer(sent, padding="max_length", max_length=512, truncation=True, return_tensors="pt") for sent in sentences]
        self.labels = [align_label(sent, label, label_to_idx=self.label_to_idx) for sent, label in zip(self.txt, labels)]
        self.len = len(self.labels)
    
    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        return self.txt[index], torch.LongTensor(self.labels[index])

In [9]:
class BertModel(torch.nn.Module):

    def __init__(self, num_unique_labels):

        super(BertModel, self).__init__()

        self.bert = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=num_unique_labels)

    def forward(self, input_id, mask, label):

        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)

        return output

In [10]:
def trainer(model):
    
    torch.cuda.empty_cache()

    tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")
    trainDataset = NERDataset(filepath= url, tokenizer=tokenizer, start=0, end=900)
    validationDataset = NERDataset(filepath= url, tokenizer=tokenizer, start=900, end=1000)
    trainDataLoader = data.DataLoader(trainDataset, batch_size=8, shuffle=True)
    valDataLoader = data.DataLoader(validationDataset, batch_size=8)
    
    cuda_available = torch.cuda.is_available() 
    device = torch.device("cuda" if  cuda_available else "cpu")
    optimizer = torch.optim.SGD(model.parameters(), lr=3e-4)
    
    if cuda_available:
        model.cuda()
    
    for epoch in range(1):
        
        train_acc, val_acc = 0, 0
        model.train()
        for train_data, train_labels in tqdm(trainDataLoader):
            train_labels = train_labels.to(device)
            input_ids = train_data["input_ids"].squeeze(1).to(device)
            attn_masks = train_data["attention_mask"].squeeze(1).to(device)
            
            optimizer.zero_grad()
            loss, logits = model(input_ids, attn_masks, train_labels)
            ### LOGITS => BATCH_SIZE * SEQ_LEN * NUM_LABELS
            ## LABELS => BATCH_SIZE * SEQ_LEN
            for idx in range(logits.shape[0]): ### iterate over all samples
                ### remove redundant tokens for accuracy computation
                clean_logits = logits[idx][train_labels[idx] != -100] ## SEQ_LEN' * NUM_LABELS
                clean_labels = train_labels[idx][train_labels[idx] != -100 ] ## SEQ_LEN'
                pred = clean_logits.argmax(dim=1) ### SEQ_LEN'
                train_acc += (pred == clean_labels).float().mean()
                
            loss.backward()
            optimizer.step()
            print(f"Train Loss: {loss.item()}", end="\r")
        
        ### EVALUATION ON VALIDATION SET
        model.eval()
        for val_data, val_labels in tqdm(valDataLoader):
            val_labels = val_labels.to(device)
            input_ids = val_data["input_ids"].squeeze(1).to(device)
            attn_masks = val_data["attention_mask"].squeeze(1).to(device)
            
            loss, logits = model(input_ids, attn_masks, val_labels)
            
            for idx in range(logits.shape[0]): ### iterate over all samples
                ### remove redundant tokens for accuracy computation
                clean_logits = logits[idx][val_labels[idx] != -100] ## SEQ_LEN' * NUM_LABELS
                clean_labels = val_labels[idx][val_labels[idx] != -100 ] ## SEQ_LEN'
                pred = clean_logits.argmax(dim=1) ### SEQ_LEN'
                val_acc += (pred == clean_labels).float().mean()
            print(f"Val Loss: {loss.item()}", end="\r")
        
        print(f"Epoch: {epoch+1} | Train Acc: {train_acc/len(trainDataset)} | Val Acc: {val_acc/len(validationDataset)}")
            

            
    
    

In [11]:
model = BertModel(num_unique_labels=17)
trainer(model)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

Number of Unique Labels: 17
Number of Unique Labels: 17


  1%|          | 1/113 [00:01<02:57,  1.59s/it]

  2%|▏         | 2/113 [00:02<01:58,  1.07s/it]

  3%|▎         | 3/113 [00:02<01:38,  1.12it/s]

...

 69%|██████▉   | 9/13 [00:02<00:01,  3.75it/s]

 77%|███████▋  | 10/13 [00:02<00:00,  3.76it/s]

 85%|████████▍ | 11/13 [00:02<00:00,  3.76it/s]

100%|██████████| 13/13 [00:03<00:00,  3.87it/s]

Epoch: 1 | Train Acc: 0.7372771501541138 | Val Acc: 0.8369261026382446


In [16]:
torch.cuda.empty_cache()

In [18]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 2            |        cudaMalloc retries: 2         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  16640 KiB |  14190 MiB |   2218 GiB |   2218 GiB |
|       from large pool |  16640 KiB |  14185 MiB |   2214 GiB |   2214 GiB |
|       from small pool |      0 KiB |      6 MiB |      4 GiB |      4 GiB |
|---------------------------------------------------------------------------|
| Active memory         |  16640 KiB |  14190 MiB |   2218 GiB |   2218 GiB |
|       from large pool |  16640 KiB |  14185 MiB |   2214 GiB |